# Análise da taxa de aprovação das disciplinas
O objetivo desse notebook é realizar uma análise da taxa de aprovação das disciplinas da UFRN, por período. 

# Os dados
Os dados que iremos utilizar foram retirados do site de [dados abertos da UFRN](http://dados.ufrn.br/). Os pacotes de dados foram:

- [Docentes](http://dados.ufrn.br/dataset/docentes)
- [Matriculas em Componentes](http://dados.ufrn.br/dataset/matriculas-componentes)
- [Turmas](http://dados.ufrn.br/dataset/turmas)

## Dependências
Para o download desses dados, iremos utilizar o pacote [odufrn-downloader](https://github.com/odufrn/odufrn-downloader).

In [4]:
!pip install -U --user odufrn_downloader

## Importações

In [1]:
import numpy as np
import pandas as pd
from os import listdir
from odufrn_downloader import ODUFRNDownloader

## Download dos arquivos

In [12]:
# Objeto
ufrn_data = ODUFRNDownloader()
# Processo de download
ufrn_data.download_packages([
    'docentes', 'matriculas-componentes', 'turmas',
], 'data', dictionary=False)

Baixando Docentes...
Baixando Dicionário de Dados - Docentes...
Baixando Matrículas de 2019.1...
Baixando Matrículas de 2018.2...
Baixando Matrículas de 2018.1...
Baixando Matrículas de 2017.2...
Baixando Matrículas de 2017.1...
Baixando Matrículas de 2016.2...
Baixando Matrículas de 2016.1...
Baixando Matrículas de 2015.2...
Baixando Matrículas de 2015.1...
Baixando Matrículas de 2014.2...
Baixando Matrículas de 2014.1...
Baixando Matrículas de 2013.2...
Baixando Matrículas de 2013.1...
Baixando Matrículas de 2012.2...
Baixando Matrículas de 2011.2...
Baixando Matrículas de 2012.1...
Baixando Matrículas de 2011.1...
Baixando Matrículas de 2010.2...
Baixando Matrículas de 2010.1...
Baixando Matrículas de 2009.2...
Baixando Matrículas de 2009.1...
Baixando Dicionário de Dados - Matrículas em Componentes...
Baixando Turmas de 2018.1...
Baixando Turmas de 2018.2...
Baixando Turmas de 2017.1...
Baixando Turmas de 2017.2...
Baixando Turmas de 2016.1...
Baixando Turmas de 2016.2...
Baixando 

## Leitura dos arquivos

In [13]:
# Dicionário que conterá os datasets
dfs = {
    'docentes': [],
    'turmas': [],
    'matriculas-componentes': [],
}

# Campos que iremos usar de cada dataset
target_columns = {
    'docentes': [
        'siape',
        'nome',
        'formacao',
    ],
    'turmas': [
        'id_turma',
        'siape',
        'id_componente_curricular',
        'ano',
        'nivel_ensino',
        'ano',
        'periodo',
        'situacao_turma',
    ],
    'matriculas-componentes': [
        'id_turma',
        'discente',
        'id_curso',
        'media_final',
        'descricao',
    ],
    
}

# Percorremos as pastas com os datasets
for package in listdir('data'):
    # Percorremos cada arquivo da pasta
    for filename in listdir('data/'+package):
        # Caminho para o arquivo
        path_file = 'data/'+package+'/'+filename
        # Lemos o dataset
        df = pd.read_csv(path_file, sep=';', usecols=target_columns[package])
        # Adiciona a coluna para identificar os períodos
        if package in ['turmas', 'matriculas-componentes']:
            df['periodo'] = filename[-10:-4]
        # Adicionamos o dataframe a lista de dataframe
        dfs[package].append(df)
        
# Ao final, concatenamos todos os datasets, gerando um só para cada pacote
for key in dfs.keys():
    dfs[key] = pd.concat(dfs[key], ignore_index=True)

A função **[pandas.DataFrame.info](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.info.html)** nos fornecerá um sumário dos dados que temos. 

In [15]:
dfs['docentes'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2361 entries, 0 to 2360
Data columns (total 3 columns):
siape       2361 non-null int64
nome        2361 non-null object
formacao    2361 non-null object
dtypes: int64(1), object(2)
memory usage: 55.5+ KB


In [16]:
dfs['turmas'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371632 entries, 0 to 371631
Data columns (total 7 columns):
id_turma                    371632 non-null int64
siape                       358806 non-null float64
id_componente_curricular    371632 non-null int64
nivel_ensino                371632 non-null object
ano                         371632 non-null int64
periodo                     371632 non-null object
situacao_turma              371632 non-null object
dtypes: float64(1), int64(3), object(3)
memory usage: 19.8+ MB


In [17]:
dfs['matriculas-componentes'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13079106 entries, 0 to 13079105
Data columns (total 6 columns):
id_turma       int64
discente       object
id_curso       float64
media_final    object
descricao      object
periodo        object
dtypes: float64(1), int64(1), object(4)
memory usage: 598.7+ MB


# Calcular a taxa de aprovação das turmas